# Computación a Gran Escala
**Curso 2022/2023**

## Lab 6: SparkSQL - Tarea 1

**Autores**:  

Miguel García González - miguel.garcia02@estudiante.uam.es  
Belén Vivas García - belen.vivas@estudiante.uam.es

### Inicialización de PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder.appName('Lab6-T1').getOrCreate()
sc = spark.sparkContext

/home/belen/spark/venv/spark/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
# Rutas de datos
PHONES_ACCELEROMETER_PATH = 'data/datos_sensores/activity_recognition_exp/Phones_accelerometer.csv'
PHONES_GYROSCOPE_PATH = 'data/datos_sensores/activity_recognition_exp/Phones_gyroscope.csv'
WATCH_ACCELEROMETER_PATH = 'data/datos_sensores/activity_recognition_exp/Watch_accelerometer.csv'
WATCH_GYROSCOPE_PATH = 'data/datos_sensores/activity_recognition_exp/Watch_gyroscope.csv'

### Funciones para gestionar DataFrames

Estas funciones nos servirán para crear DataFrames, manipularlos y unirlos.

In [4]:
"""
Function: create_DF

Params:
    - filename: fichero CSV sobre el que se va a crear un DataFrame

Return:
    DataFrame creado

Esta funcion recibe un fichero CSV y devuelve un DataFrame. Se encarga de crear la cabecera con
el nombre de los campos.
"""
def create_DF(filename):
    
    file = sc.textFile(filename).map(lambda e: e.split(","))
    file_header = file.map(lambda e: Row(index=e[0],
                                         arrival_time=e[1],
                                         creation_time=e[2],
                                         x=float(e[3]),
                                         y=float(e[4]),
                                         z=float(e[5]),
                                         user=e[6],
                                         model=e[7],
                                         device=e[8],
                                         gt=e[9]))
    df = spark.createDataFrame(file_header)
    
    return df


"""
Function: process_DF

Params:
    - df: DataFrame a procesar

Return:
    DataFrame procesado

Esta funcion transforma el DataFrame a la forma:
User | Model | GT | mean(x) | mean(y) | mean(z) | std(x) | std(y) | std(z) |
max(x) | max(y) | max(z) | min(x) | min(y) | min(z)
reduciendo primero el DataFrame unicamente con los campos necesarios, agrupando despues en
distintos DataFrames la clave primaria con la media, desviacion tipica, maximo y minimo y,
finlamente, uniendo cada uno de ellos de nuevo.
"""
def process_DF(df):
    
    # Nos quedamos con los campos que nos interesan
    reduced_df = df.select(df['user'], df['model'], df['gt'], df['x'], df['y'], df['z'])
    
    # Agrupamos por clave y hacemos cada uno de los calculos
    mean_df = reduced_df.groupBy('user', 'model', 'gt').agg(f.mean('x'), 
                                                            f.mean('y'), 
                                                            f.mean('z'))
    
    stdev_df = reduced_df.groupBy('user', 'model', 'gt').agg(f.stddev_pop('x'), 
                                                             f.stddev_pop('y'), 
                                                             f.stddev_pop('z'))
    
    max_df = reduced_df.groupBy('user', 'model', 'gt').agg(f.max('x'), 
                                                           f.max('y'), 
                                                           f.max('z'))
    
    min_df = reduced_df.groupBy('user', 'model', 'gt').agg(f.min('x'), 
                                                           f.min('y'),
                                                           f.min('z'))
    
    # Juntamos los resultados
    processed_df = mean_df.join(
        stdev_df, on=['user', 'model', 'gt']).join(
        max_df, on=['user', 'model', 'gt']).join(
        min_df, on=['user', 'model', 'gt'])

    return processed_df


"""
Function: join_DF

Params:
    - pa_df: DataFrame de Phones accelerometer
    - pg_df: DataFrame de Phones gyroscope
    - wa_df: DataFrame de Watch accelerometer
    - wg_df: DataFrame de Watch gyroscope

Return:
    DataFrame final

Esta funcion concatena los registros de los telefonos por un lado y de los relojes por otro.
Una vez hecho esto, los une en un DataFrame final.
"""
def join_DF(pa_df, pg_df, wa_df, wg_df):
    
    # Unimos phones por un lado y watches por otro mediante join
    phones_df = pa_df.join(pg_df, on=['user', 'model', 'gt'])
    watch_df = wa_df.join(wg_df, on=['user', 'model', 'gt'])

    # Unimos todos mediante union
    final_df = phones_df.union(watch_df)
    
    return final_df

### DataFrames

En primer lugar, creamos los cuatro DataFrames a partir de los ficheros CSV proporcionados:

In [5]:
pa_df = create_DF(PHONES_ACCELEROMETER_PATH)
pg_df = create_DF(PHONES_GYROSCOPE_PATH)
wa_df = create_DF(WATCH_ACCELEROMETER_PATH)
wg_df = create_DF(WATCH_GYROSCOPE_PATH)

Los procesamos para obtenerlos con el formato deseado y agrupados por clave primaria con la media, desviación típica, máximo y mínimo de sus variables:

In [6]:
processed_pa_df = process_DF(pa_df)
processed_pg_df = process_DF(pg_df)
processed_wa_df = process_DF(wa_df)
processed_wg_df = process_DF(wg_df)

Finalmente, unimos los registros y creamos un DataFrame único.

In [7]:
final_df = join_DF(processed_pa_df, processed_pg_df, processed_wa_df, processed_wg_df)
final_df.show()

+----+------+----------+--------------------+--------------------+------------------+-------------------+--------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+
|user| model|        gt|              avg(x)|              avg(y)|            avg(z)|      stddev_pop(x)|       stddev_pop(y)|      stddev_pop(z)|             max(x)|            max(y)|            max(z)|             min(x)|             min(y)|            min(z)|              avg(x)|              avg(y)|              avg(z)|       stddev_pop(x)|       stddev_pop(y)|       stddev_pop(z)|            max(x)|            max(y)|            max(z)|             min(x)|             min(y)|       